# Training for Protovest

In [1]:
%matplotlib inline

from pathlib import Path
from itertools import product
from io import StringIO
from contextlib import redirect_stdout
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown, Latex

## Load dataset

In [2]:
dfs = {}
dataset_dir = Path("../datasets/Thrust 2 Data Collections/Thrust 2 IMU/Round 1/with orientation")
total_samples = 0
for data_file in dataset_dir.glob("*.csv"):
    print(f"Reading {data_file} :", end="")
    dfs[data_file.stem] = pd.read_csv(data_file)
    n_samples = dfs[data_file.stem].shape[0]
    print(f"{n_samples:_} samples")
    total_samples += n_samples
    
print(f"{total_samples = :_}")

Reading ../datasets/Thrust 2 Data Collections/Thrust 2 IMU/Round 1/with orientation/Trial5.csv :450_464 samples
Reading ../datasets/Thrust 2 Data Collections/Thrust 2 IMU/Round 1/with orientation/Trial2.csv :310_096 samples
Reading ../datasets/Thrust 2 Data Collections/Thrust 2 IMU/Round 1/with orientation/Trial4.csv :412_589 samples
Reading ../datasets/Thrust 2 Data Collections/Thrust 2 IMU/Round 1/with orientation/Trial3.csv :357_880 samples
Reading ../datasets/Thrust 2 Data Collections/Thrust 2 IMU/Round 1/with orientation/Trial1.csv :704_736 samples
total_samples = 2_235_765


## Define study and CV experiments

In [3]:
def get_features(locations, sensors):
    return list(map("_".join, product(locations, sensors)))

    
def cv_experiment_gen(features, target="Mode"):        
    for test_trial in dfs.keys():
        train_seqs = []
        for trial_name, trial_df in dfs.items():
            X, y = trial_df.loc[:, features].values, trial_df.loc[:, target].values
            if trial_name != test_trial:
                train_seqs.append((X, y))
            else:
                test_seq = (X, y)
                
        yield test_trial, train_seqs, test_seq

    
def all_experiment_gen(features, target="Mode"):        
    seqs = []
    for trial_name, trial_df in dfs.items():
        X, y = trial_df.loc[:, features].values, trial_df.loc[:, target].values
        seqs.append((X, y))
    
    # TODO / HACK: we are just using the first seq for testing here
    yield "all", seqs, seqs[0]

#experiment_gen = cv_experiment_gen
experiment_gen = all_experiment_gen
        
loc_trunk = ["Trunk"]

sens_q = ["Orientation_q0", "Orientation_q1", "Orientation_q2", "Orientation_q3"]
sens_euler = ["EulerX", "EulerY", "EulerZ"]
sens_accel = ["Accel_X", "Accel_Y", "Accel_Z"]
sens_gyro = ["Gyro_X", "Gyro_Y", "Gyro_Z"]


#
# STUDY CONFIGS
# 

study_name = "raw"
study_features = get_features(loc_trunk, sens_gyro + sens_accel)


with redirect_stdout(StringIO()) as info:
    print(f"{study_name = }")
    print(f"{study_features = }")
    for experiment, train_seqs, test_seq in experiment_gen(study_features):
        print(f"{experiment}: {list(map(lambda s: s[0].shape, train_seqs))} -> {test_seq[0].shape}")

print(info.getvalue())

results_dir = Path("./results") / Path(str(dataset_dir.name) + f" - {study_name}")
results_dir.mkdir(parents=True, exist_ok=True)

with open(results_dir / "info.txt", "w") as info_file:
    info_file.write(info.getvalue())

study_name = 'raw'
study_features = ['Trunk_Gyro_X', 'Trunk_Gyro_Y', 'Trunk_Gyro_Z', 'Trunk_Accel_X', 'Trunk_Accel_Y', 'Trunk_Accel_Z']
all: [(450464, 6), (310096, 6), (412589, 6), (357880, 6), (704736, 6)] -> (450464, 6)



## Model

In [4]:
class RNNModel(nn.Module):
    # X: (<1>, seq, features)
    # h0, c0: (num_layers, 1, hidden_size)
    # out: (<1>, seq)
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        #self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # or:
        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.reset_hidden()
        
    def forward(self, x):
        #out, h0 = self.rnn(x, self.h0)  
        # or:
        out, (h0, c0) = self.lstm(x, (self.h0, self.c0))
        out = self.fc(out)
        self.h0, self.c0 = h0.detach(), c0.detach()
        return out
    
    def reset_hidden(self):
        self.h0 = torch.zeros(self.num_layers, 1, self.hidden_size)
        self.c0 = torch.zeros(self.num_layers, 1, self.hidden_size) 

# model = RNNModel(train_seqs[0][0].shape[-1], hidden_size=16, num_layers=2)

## Build trained model

In [5]:
def build_model(train_seqs, num_epochs=1, batch_len=256):
    # Model
    model = RNNModel(train_seqs[0][0].shape[-1], hidden_size=16, num_layers=2)
    
    # Loss and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    # Training
    model.train()
    epoch_steps = sum(X.shape[0] for X, _ in train_seqs) // batch_len
    for epoch in range(num_epochs):
        epoch_total_loss = 0
        epoch_total_accuracy = 0
        epoch_samples = 0
        with tqdm(desc=f"Epoch {epoch}", total=epoch_steps) as pbar:
            for seq_X, seq_y in train_seqs:
                model.reset_hidden()
                for i in range(0, seq_X.shape[0], batch_len):
                    # add dummy batch dimension
                    X = torch.from_numpy(seq_X[np.newaxis, i : i + batch_len]).float()
                    # add dummy batch and target dimensions
                    y = torch.from_numpy(seq_y[np.newaxis, i : i + batch_len, np.newaxis] ).float()
                    output = model(X)
                    loss = criterion(output, y)

                    # Backward and optimize
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    epoch_total_loss += loss.item()
                    y_hat = torch.where(output.detach() < 0, 0, 1)
                    epoch_total_accuracy += torch.sum(y_hat == y).item()

                    epoch_samples += X.shape[1]
                    pbar.update()
                    if epoch_samples and epoch_samples % 100 == 0:
                        pbar.set_postfix(loss=epoch_total_loss/epoch_samples,
                                         accuracy=epoch_total_accuracy/epoch_samples)
                        
    return model

#model = build_model(train_seqs)

## Test predictions

In [6]:
def hysteresis(x, th_lo, th_hi):
    initial = x[0] > th_hi
    hi = x >= th_hi
    lo_or_hi = (x <= th_lo) | hi
    ind = np.nonzero(lo_or_hi)[0]
    if not ind.size: # prevent index error if ind is empty
        return np.zeros_like(x, dtype=bool) | initial
    cnt = np.cumsum(lo_or_hi) # from 0 to len(x)
    return np.where(cnt, hi[ind[cnt-1]], initial)


def model_predictions(model, test_seq, batch_len=256):
    model.eval()
    X, _ = test_seq
    with torch.no_grad():
        model.reset_hidden()
        y = []
        for i in range(0, X.shape[0], batch_len):
            # add dummy batch dimension
            X_batch = torch.from_numpy(X[np.newaxis, i : i + batch_len]).float()
            y.append(model(X_batch).view(-1))

        soft_preds = torch.sigmoid(torch.cat(y)).numpy()
        hard_preds = soft_preds > 0.5
        hyst_preds = hysteresis(soft_preds, 0.2, 0.8)

    return hard_preds, soft_preds, hyst_preds

#hard, soft, hyst = model_predictions(model, test_seq)

## Evaluation

In [7]:
def evaluate(experiment, test_seq, hard, soft, hyst):

    import matplotlib.pyplot as plt
    plt.rcParams['axes.labelsize'] = 16
    plt.rcParams['axes.titlesize'] = 18

    _, y = test_seq
    

    plt.figure(figsize=(16,10))
    
    ax = plt.subplot(2, 1, 1)
    ax.title.set_text(f"Clutch Control Timeline: {experiment}")
    
    plt.plot(y, "k", label="Target");
    plt.plot(hard + 0.01, "b", label="Hard prediction", alpha=0.3);
    plt.plot(soft + 0.02, "g", label="Soft Predictions", alpha=0.25);
    plt.plot(hyst + 0.03, "r", label="Hysteresis Predictions", alpha=0.5);

    plt.legend()
    plt.yticks([0, 1], ["off", "on"])
    plt.xlabel("Time (sample)")
    
    
    ax = plt.subplot(2, 2, 3)
    ax.title.set_text(f"Hard Predictions: {(y == hard).mean():.2%}")
    cf_matrix = confusion_matrix(y, hard)
    f = sns.heatmap(cf_matrix / cf_matrix.sum(), annot=True, fmt='.2%', annot_kws={"size": 18})
    plt.xlabel("Prediction")
    plt.ylabel("Target")

    ax = plt.subplot(2, 2, 4)
    ax.title.set_text(f"Hysteresis Predictions: {(y == hyst).mean():.2%}")
    cf_matrix = confusion_matrix(y, hyst)
    f = sns.heatmap(cf_matrix / cf_matrix.sum(), annot=True, fmt='.2%', annot_kws={"size": 18})
    plt.xlabel("Prediction")
    plt.ylabel("Target")

    plt.tight_layout()
    plt.savefig(results_dir / f"{experiment}.png")
    plt.show()
    
    result_df = dfs[experiment]
    result_df["pred_hard"] = hard.astype(int)
    result_df["pred_soft"] = soft
    result_df["pred_hyst"] = hyst.astype(int)
    result_df.to_csv(results_dir / f"{experiment}.csv", index=False)

#evaluate(experiment, test_seq, hard, soft, hyst)

## Execute experiments

In [ ]:
targets, hards, hysts = [], [], []
for experiment, train_seqs, test_seq in experiment_gen(study_features):
    display(Markdown(f"### Experiment: {experiment}"))
    display(Markdown(f"```\n{list(map(lambda s: s[0].shape, train_seqs))} -> {test_seq[0].shape}\n```"))
    
    model = build_model(train_seqs)
    torch.save(model.state_dict(), results_dir / "state_dict.pt")
    torch.jit.script(model).save(results_dir / "model_scripted.pt")
    hard, soft, hyst = model_predictions(model, test_seq)
    evaluate(experiment, test_seq, hard, soft, hyst)
    targets.append(test_seq[1])
    hards.append(hard)
    hysts.append(hyst)
    
target = np.concatenate(targets)
hard = np.concatenate(hards)
hyst = np.concatenate(hysts)

plt.figure(figsize=(16,6))
ax = plt.subplot(1, 2, 1)
ax.title.set_text(f"Hard Predictions: {(target == hard).mean():.2%}")
cf_matrix = confusion_matrix(target, hard)
f = sns.heatmap(cf_matrix / cf_matrix.sum(), annot=True, fmt='.2%', annot_kws={"size": 18})
plt.xlabel("Prediction")
plt.ylabel("Target")

ax = plt.subplot(1, 2, 2)
ax.title.set_text(f"Hysteresis Predictions: {(target == hyst).mean():.2%}")
cf_matrix = confusion_matrix(target, hyst)
f = sns.heatmap(cf_matrix / cf_matrix.sum(), annot=True, fmt='.2%', annot_kws={"size": 18})
plt.xlabel("Prediction")
plt.ylabel("Target")

#plt.title("Overall Results")
plt.savefig(results_dir / f"overall.png")

### Experiment: all

```
[(450464, 6), (310096, 6), (412589, 6), (357880, 6), (704736, 6)] -> (450464, 6)
```

Epoch 0:  44%|█████████████████▌                      | 3831/8733 [03:32<04:28, 18.25it/s, accuracy=0.735, loss=0.00209]